# **Movie Recommendation System**

---

In [1]:
import pandas as pd
import numpy as np

In [2]:
movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")
movies_list = pd.read_csv("movies.csv")
ratings = pd.read_csv("ratings.csv")

In [3]:
ratings.head(1)

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044


In [4]:
movies_list.head(1)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [5]:
movies_list = movies_list.drop('genres', axis=1)
movies_list.head(1)

,movieId,title
0,1,Toy Story (1995)


In [6]:
movies_list['title'] = movies_list['title'].str.replace(r'\(\d{4}\)', '', regex=True).str.strip()
movies_list.head(2)

,movieId,title
0,1,Toy Story
1,2,Jumanji


In [7]:
ratings_with_movies = ratings.merge(movies_list, on='movieId')
ratings_with_movies.head(2)

,userId,movieId,rating,timestamp,title
0,1,296,5.0,1147880044,Pulp Fiction
1,3,296,5.0,1439474476,Pulp Fiction


In [8]:
ratings_with_movies.shape

(25000095, 5)

In [9]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [10]:
movies = movies[['genres', 'id', 'keywords', 'overview', 'title']]
movies.head(1)

,genres,id,keywords,overview,title
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",Avatar


In [11]:
movies = movies.merge(movies_list, on='title')
movies.head(1)

,genres,id,keywords,overview,title,movieId
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",Avatar,72998


In [12]:
movies.shape

(4323, 6)

In [13]:
ratings_with_movies = ratings_with_movies.drop_duplicates(subset=['userId', 'movieId', 'rating'])
ratings_with_movies.shape

(25000095, 5)

In [14]:
movies_ratings_all = movies.merge(ratings, on = 'movieId')
movies_ratings_with_newID = movies_ratings_all[['userId', 'id', 'title', 'rating']]
movies_ratings_with_newID.head(1)

,userId,id,title,rating
0,3,19995,Avatar,4.0


In [15]:
movies_ratings_with_newID.shape

(12680181, 4)

In [16]:
movies_ratings_with_newID = movies_ratings_with_newID.drop_duplicates(subset=['userId', 'id', 'rating'])
movies_ratings_with_newID.shape

(12664758, 4)

#### At this Point we have only two dataframe
    1. movies ratings with changed movie ID
    2. movies details

In [17]:
rating_main = movies_ratings_with_newID
rating_main.head(1)

,userId,id,title,rating
0,3,19995,Avatar,4.0


In [23]:
rating_main.shape

(12664758, 4)

In [20]:
# movies = movies.drop('movieId', axis=1)
movies.head(1)
# movies.shape

,genres,id,keywords,overview,title
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",Avatar


DATA PREPARATION DONE....

In [21]:
x = rating_main.groupby('userId').count()['rating'] > 200
genuine_user = x[x].index

In [22]:
filtered_rating = rating_main[rating_main['userId'].isin(genuine_user)]
filtered_rating.shape

(5363482, 4)

In [25]:
y = filtered_rating.groupby('title').count()['rating']>=50
famous_movies = y[y].index

In [27]:
final_ratings = filtered_rating[filtered_rating['title'].isin(famous_movies)]
final_ratings.shape

(5354017, 4)

In [29]:
user_movie_rating_table = final_ratings.pivot_table(index='title',columns='userId',values='rating')

In [30]:
user_movie_rating_table.fillna(0, inplace=True)

In [31]:
user_movie_rating_table

userId,3,12,13,31,43,57,72,75,80,120,...,162481,162484,162495,162508,162512,162516,162519,162521,162533,162534
title,,,,,,,,,,,,,,,,,,,,,
(500) Days of Summer,0.0,3.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,...,0.0,0.0,0.0,2.5,0.0,4.0,0.0,0.0,0.0,0.0
10 Cloverfield Lane,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0
10 Things I Hate About You,0.0,3.0,0.0,0.0,0.0,0.0,5.0,3.0,4.0,5.0,...,0.0,3.0,2.0,3.0,5.0,3.5,0.0,0.0,2.5,0.0
102 Dalmatians,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
11:14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,3.5,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
[REC],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.5,0.0,3.5,0.0,0.0,0.0,0.0
[REC]²,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
eXistenZ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,4.0,2.5,0.0,0.0,0.0,3.5,3.0,4.0,0.0,0.0


In [32]:
final_ratings[final_ratings['title']=='Avatar']

,userId,id,title,rating
0,3,19995,Avatar,4.0
2,12,19995,Avatar,4.0
3,13,19995,Avatar,4.0
11,75,19995,Avatar,3.0
20,181,19995,Avatar,1.5
...,...,...,...,...
24541,162508,19995,Avatar,5.0
24542,162512,19995,Avatar,4.0
24543,162516,19995,Avatar,4.0
24545,162533,19995,Avatar,3.0


In [33]:
from sklearn.metrics.pairwise import cosine_similarity

In [35]:
similarity_scores = cosine_similarity(user_movie_rating_table)
similarity_scores.shape

(2900, 2900)